In [ ]:
from typing import Type
from BLUEPRINT.base import ParameterFrame, ReactorSystem, TypeFrameworkError
from BLUEPRINT.geometry.loop import Loop

# Anatomy of a ReactorSystem

In [ ]:
class TypicalSystem(ReactorSystem):
    """
    The class docstring. It should contain a simple description of what
    the class is.
    """

    # This is where we define class attributes (we can give type-hints)

    name: str
    config: Type[ParameterFrame]
    inputs: dict

    something: Type[Loop]

    # Here we define the Parameters that the TypicalSystem class will use
    # These will be pulled from the Reactor that we are making when using this
    # class

    # fmt: off  --> this is the flag for black to ignore this part of the code
    default_params = [
        ["n_TF", "Number of TF coils", 16, "N/A", None, "Input"],
        ["coolant", "Coolant", "Water", None, "Divertor coolant type", "Common sense"],
        ["T_in", "Coolant inlet T", 80, "°C", "Coolant inlet T", None],
    ]
    # fmt: on   --> turn formatting back on!

    def __init__(self, config: Type[ParameterFrame], inputs: dict) -> None:
        self.config = config
        self.inputs = inputs

        # Here are going to update the default Parameters with the config
        # (which normally comes from the Reactor)
        self.params = ParameterFrame(self.default_params.to_records())
        self.params.update_kw_parameters(self.config)

    def add_loop(self, loop: Type[Loop]) -> None:
        """
        Assign a loop to the class.

        Parameters
        ----------
        loop: Loop
            The Loop to use
        """
        self.something = loop

Now let's make an instance of the TypicalSystem

First, we make a ParameterFrame, which we will make really small here, but in
practice will come from a Reactor, and have lots of things in it

In [ ]:
config = ParameterFrame(
    [
        ["n_TF", "Number of TF coils", 18, "N/A", None, "Input"],
        ["n_PF", "Number of PF coils", 8, "N/A", None, "Input"],
    ]
)
inputs = {"something": "random"}


typ_system = TypicalSystem(config, inputs)

Notice 3 things:
*  the class default for n_TF was 16, and we passed a config with 18.
   Now the TypicalSystem thinks n_TF = 18
*  The config ParameterFrame had a Parameter n_PF, which wasn't defined
   in the TypicalSystem class. TypicalSystem ignores this Parameter: it
   simply doesn't update its ParameterFrame
*  Some Parameters were in TypicalSystem, that were not updated by our
   "global" config. They are left untouched.

In [ ]:
print(typ_system.params)

OK now let's add a Loop to our TypicalSystem

In [ ]:
loop = Loop(x=[1, 2, 3, 4, 5, 1], y=[0, 1, 2, 3, 0, 0])

typ_system.add_loop(loop)

So far, so good. But what about all these type annotations?
Python is dynamically typed, and sometimes this can cause problems, in
particular when doing a lot of very different things...
Using some magic in the base module, our type annotations are now strictly
enforced by default.

So let's try and break things...

In [ ]:
try:
    typ_system.name = 42
except TypeFrameworkError as e:
    print(e)

In [ ]:
try:
    typ_system.add_loop(34.6)
except TypeFrameworkError as e:
    print(e)

In [ ]:
try:
    typ_system.something = "nothing"
except TypeFrameworkError as e:
    print(e)